In [1]:
import math
import os

import numpy as np
import pandas as pd
import torch

SEED = 3

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [2]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

In [3]:
from datetime import date

from torchvision import transforms

from util.dataset import PassengerFlowDataset
from util.transform import PandasToTensor, RollExogenousFeatures

transform = transforms.Compose([
    PandasToTensor(),
    RollExogenousFeatures()
])

train_data = PassengerFlowDataset(
    min_date=date(2022, 1, 1),
    max_date=date(2023, 1, 1),
    transform=transform)
validation_data = PassengerFlowDataset(
    min_date=date(2023, 1, 1),
    max_date=date(2023, 4, 1),
    transform=transform)
test_data = PassengerFlowDataset(
    min_date=date(2023, 4, 1),
    max_date=date(2023, 7, 1),
    transform=transform)

train_data._data

LOADING DATA: 100%|██████████| 6/6 [00:07<00:00,  1.26s/it]


passengers     noise
datetime            origin destination                      
2022-01-01 00:00:00 12     19                159.0  1.788628
                           LM                  6.0  0.436510
                           OW                 25.0  0.096497
                    16     24                 78.0 -1.863493
                           CC                 82.0 -0.277388
...                                            ...       ...
2022-12-31 23:00:00 WD     ED                 17.0 -1.369044
                    WP     NC                 14.0 -1.853796
                           PC                 35.0 -1.561378
                    WS     FM                121.0 -0.871452
                           ML                 16.0  1.432205

[893520 rows x 2 columns]

In [4]:
from torch.utils.data import DataLoader

BATCH_SIZE = 256

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
validation_loader = DataLoader(validation_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [5]:
from torch.optim.lr_scheduler import ExponentialLR
from models.sarima import SARIMA
from torch.optim import AdamW

import torch.nn as nn

device = torch.device('cuda:0')

model = SARIMA(order=(23, 1, 0),
               seasonal_lag=24,
               seasonal_order=(7, 0, 0)
               ).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=1.0e-3, eps=1.0e-4)
scheduler = ExponentialLR(optimizer, 0.9)

In [ ]:
import wandb

# Log run to Weights and Biases
run = wandb.init(
    project='passenger-flow-forecasting',
    config={
        'model': 'SARIMA',
        'p': model._order[0],
        'd': model._order[1],
        'q': model._order[2],
        'P': model._seasonal_order[0],
        'D': model._seasonal_order[1],
        'Q': model._seasonal_order[2]
    })

In [7]:
from datetime import datetime

from sklearn import metrics
from tqdm.notebook import tqdm

EPOCHS = 10
last_state = (-1, None)

# Create weights directories
os.makedirs('weights/sarima/checkpoints', exist_ok=True)

tqdm_epoch = tqdm(desc='EPOCH', total=EPOCHS)
tqdm_batch = tqdm(desc='TRAIN', total=0)

for epoch in range(EPOCHS):
    # Set the model to training mode
    model.train()

    # Keep track of the training loss
    y_true = []
    y_pred = []

    # Loop over training data in batches
    tqdm_batch.reset(len(train_loader))
    tqdm_batch.desc = 'TRAIN'
    for batch in train_loader:
        # Move the data to the same device as the model
        history, horizon = tuple(t.to(device) for t in batch)

        # Select views of data
        y = horizon[:, 0, 0].squeeze()

        # Clear the gradients
        optimizer.zero_grad()

        # Compute outputs using a forward pass
        outputs = model(history, horizon).squeeze()

        # Compute the training loss of this batch
        loss = criterion(outputs, y)

        # Perform a backward pass to update weights
        loss.backward()
        optimizer.step()

        # Keep track of training loss
        y_true += y.cpu().numpy().tolist()
        y_pred += outputs.cpu().detach().numpy().tolist()

        tqdm_batch.update()

    train_mse = metrics.mean_squared_error(y_true, y_pred)
    train_mae = metrics.mean_absolute_error(y_true, y_pred)
    y_true = []
    y_pred = []

    # We don't need to keep track of gradients while testing on the validation set
    with torch.no_grad():
        model.eval()

        # Loop over data in batches
        tqdm_batch.reset(len(validation_loader))
        tqdm_batch.desc = 'VALIDATE'
        for batch in validation_loader:
            # Move the data to the same device as the model
            history, horizon = tuple(t.to(device) for t in batch)

            # Select views of data
            y = horizon[:, 0, 0].squeeze()

            # Compute outputs using a forward pass
            outputs = model(history, horizon).squeeze()

            # Keep track of validation loss
            y_true += y.cpu().numpy().tolist()
            y_pred += outputs.cpu().detach().numpy().tolist()

            tqdm_batch.update()

    validation_mse = metrics.mean_squared_error(y_true, y_pred)
    validation_mae = metrics.mean_absolute_error(y_true, y_pred)

    print(f'#{epoch:2d}    RMSE: {math.sqrt(validation_mse):.2f}    MAE: {validation_mae:.2f}')

    # Log metrics to Weights and Biases
    wandb.log({
        'train_mse': train_mse,
        'train_mae': train_mae,
        'validation_mse': validation_mse,
        'validation_mae': validation_mae
    }, commit=epoch < EPOCHS - 1)

    scheduler.step()

    last_state = epoch, model.state_dict()

    if epoch < 5 or epoch % 5 == 0:
        torch.save(model.state_dict(), f'weights/sarima/checkpoints/{epoch:2d}.pt')

    tqdm_epoch.update()

tqdm_batch.close()
tqdm_epoch.close()

EPOCH:   0%|          | 0/10 [00:00<?, ?it/s]

TRAIN: 0it [00:00, ?it/s]

# 0    RMSE: 139.97    MAE: 73.45
# 1    RMSE: 128.53    MAE: 59.83
# 2    RMSE: 127.28    MAE: 57.77
# 3    RMSE: 128.66    MAE: 58.52
# 4    RMSE: 128.44    MAE: 59.29
# 5    RMSE: 128.58    MAE: 58.48
# 6    RMSE: 129.14    MAE: 58.47
# 7    RMSE: 128.41    MAE: 58.50
# 8    RMSE: 129.28    MAE: 58.21
# 9    RMSE: 128.68    MAE: 58.45


In [8]:
# Save the trained model
now = datetime.now()
datestring = f'{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}-{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}'
torch.save(last_state[1], f'weights/sarima/{datestring}--{last_state[0]}.pt')
torch.save(last_state[1], f'weights/sarima/seed-{SEED}.pt')
torch.save(last_state[1], f'weights/sarima/latest.pt')

In [9]:
from os.path import exists

if exists(f'weights/sarima/latest.pt'):
    model.load_state_dict(torch.load(f'weights/sarima/latest.pt'))

In [10]:
import math

# Keep track of the loss
y_true = []
y_pred = []

# We don't need to keep track of gradients while testing
with torch.no_grad():
    model.eval()

    # Loop over data in batches
    for batch in tqdm(test_loader, desc='TEST'):
        # Move the data to the same device as the model
        history, horizon = tuple(t.to(device) for t in batch)

        # Select views of data
        y = horizon[:, 0, 0].squeeze()

        # Compute outputs using a forward pass
        outputs = model(history, horizon).squeeze()

        # Keep track of loss
        y_true += y.cpu().numpy().tolist()
        y_pred += outputs.cpu().detach().numpy().tolist()

        tqdm_batch.update()

test_mse = metrics.mean_squared_error(y_true, y_pred)
test_rmse = metrics.mean_squared_error(y_true, y_pred, squared=False)
test_mae = metrics.mean_absolute_error(y_true, y_pred)
test_mape = metrics.mean_absolute_percentage_error(np.array(y_true) + 1, np.array(y_pred) + 1)

# Log metrics to Weights and Biases
wandb.log({
    'mse': test_mse,
    'rmse': test_rmse,
    'mae': test_mae,
    'mape': test_mape
}, commit=True)

print(f'MSE: {test_mse:.2f}')
print(f'RMSE: {test_rmse:.2f}')
print(f'MAE: {test_mae:.2f}')
print(f'MAPE: {test_mape:.2f}')

TEST:   0%|          | 0/775 [00:00<?, ?it/s]

MSE: 18154.36
RMSE: 134.74
MAE: 60.77
MAPE: 2.79


In [ ]:
wandb.finish()